<a href="https://colab.research.google.com/github/SathyaSudha-96/Erdos/blob/main/SBM_16Layer_ClusterDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install_anaconda()

✨🍰✨ Everything looks OK!


## Load and create environment

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%pwd

'/content'

In [4]:
# Execution time 4m 18s
%cp -av /content/drive/MyDrive/'Colab Notebooks'/'Colab Notebooks'/benchmarking-gnns /content

'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/benchmarking-gnns/main_superpixels_graph_classification.py' -> '/content/benchmarking-gnns/main_superpixels_graph_classification.py'
'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/benchmarking-gnns/main_CSL_graph_classification.py' -> '/content/benchmarking-gnns/main_CSL_graph_classification.py'
'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/benchmarking-gnns/main_superpixels_graph_classification.ipynb' -> '/content/benchmarking-gnns/main_superpixels_graph_classification.ipynb'
'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/benchmarking-gnns/.gitignore' -> '/content/benchmarking-gnns/.gitignore'
'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/benchmarking-gnns/main_molecules_graph_regression.py' -> '/content/benchmarking-gnns/main_molecules_graph_regression.py'
'/content/drive/MyDrive/Colab Notebooks/Colab Notebooks/benchmarking-gnns/environment_cpu.yml' -> '/content/benchmarking-gnns/environment

In [5]:
%cd benchmarking-gnns

/content/benchmarking-gnns


In [6]:
%cd /content/benchmarking-gnns/

/content/benchmarking-gnns


In [7]:
!conda env create -f /content/benchmarking-gnns/environment_gpu.yml


CondaValueError: prefix already exists: /usr/local/envs/benchmark_gnn



In [8]:
!conda activate benchmark_gnn


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




## Main Driver Notebook for Training Graph NNs on SBM Datasets

### MODELS
- GatedGCN 
- GCN 
- GAT 
- GraphSage 
- MLP
- GIN
- MoNet
- RingGNN  
- 3WLGNN  

### DATASET
- SBM_CLUSTER
- SBM_PATTERN

### TASK
- Node Classification 

In [9]:
#!pip install tensorboardX

In [10]:
!pip install dgl-cu101
#!pip install tensorboardX
!pip install pytorch
#!pip install dgl
!pip install tensorboardX


  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/local/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-b4bwcg_v/pytorch/setup.py'"'"'; __file__='"'"'/tmp/pip-install-b4bwcg_v/pytorch/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-rlm61lyx/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7m/pytorch Check the logs for full command output.


In [11]:

import dgl
import numpy as np
import os
import socket
import time
import random
import glob
import argparse, json
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

from tensorboardX import SummaryWriter
from tqdm import tqdm

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

Using backend: pytorch


In [12]:
#import dgl

In [13]:
# """
#     AUTORELOAD IPYTHON EXTENSION FOR RELOADING IMPORTED MODULES
# """

def in_ipynb():
    try:
        cfg = get_ipython().config 
        return True
    except NameError:
        return False
    
notebook_mode = in_ipynb()
print(notebook_mode)

if notebook_mode == True:
    %load_ext autoreload
    %autoreload 2

True


In [14]:
"""
    IMPORTING CUSTOM MODULES/METHODS
"""

from nets.SBMs_node_classification.load_net import gnn_model # import GNNs
from data.data import LoadData # import dataset


In [15]:
"""
    GPU Setup
"""
def gpu_setup(use_gpu, gpu_id):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)  

    if torch.cuda.is_available() and use_gpu:
        print('cuda available with GPU:',torch.cuda.get_device_name(0))
        device = torch.device("cuda")
    else:
        print('cuda not available')
        device = torch.device("cpu")
    return device


# select GPU or CPU
use_gpu = True; gpu_id = 0; device = None # default GPU
#use_gpu = False; gpu_id = -1; device = None # CPU

In [16]:
# """
#     USER CONTROLS
# """
if notebook_mode == True:
    
    #MODEL_NAME = '3WLGNN'
    #MODEL_NAME = 'RingGNN'
    MODEL_NAME = 'GatedGCN'
    #MODEL_NAME = 'GCN'
    #MODEL_NAME = 'GAT'
    #MODEL_NAME = 'GraphSage'
    #MODEL_NAME = 'MLP'
    #MODEL_NAME = 'GIN'
    #MODEL_NAME = 'MoNet'
    

    DATASET_NAME = 'SBM_CLUSTER'
    #DATASET_NAME = 'SBM_PATTERN'

    out_dir = 'out/SBMs_node_classification/'
    root_log_dir = out_dir + 'logs/' + MODEL_NAME + "_" + DATASET_NAME + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    root_ckpt_dir = out_dir + 'checkpoints/' + MODEL_NAME + "_" + DATASET_NAME + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')

    dataset = LoadData(DATASET_NAME)
    trainset, valset, testset = dataset.train, dataset.val, dataset.test


[I] Loading dataset SBM_CLUSTER...
train, test, val sizes : 10000 1000 1000
[I] Finished loading.
[I] Data load time: 12.6088s


In [17]:
#MODEL_NAME = 'RingGNN'
MODEL_NAME = 'GatedGCN'
#MODEL_NAME = 'GCN'
#MODEL_NAME = 'GAT'
#MODEL_NAME = 'GraphSage'
#MODEL_NAME = 'MLP'
#MODEL_NAME = 'DiffPool'
#MODEL_NAME = 'GIN'
#MODEL_NAME = 'MoNet'


In [18]:
# """
#     PARAMETERS
# """
if notebook_mode == True:

    n_heads = -1
    edge_feat = False
    pseudo_dim_MoNet = -1
    kernel = -1
    gnn_per_block = -1
    embedding_dim = -1
    pool_ratio = -1
    n_mlp_GIN = -1
    gated = False
    self_loop = False
    #self_loop = True
    max_time = 12
    pos_enc = True
    #pos_enc = False
    pos_enc_dim = 10
    

    if MODEL_NAME == 'GatedGCN':
        seed=41; epochs=1000; batch_size=5; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=16; hidden_dim=70; out_dim=hidden_dim; dropout=0.0; readout='mean'

    if MODEL_NAME == 'GCN':
        seed=41; epochs=1000; batch_size=5; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=146; out_dim=hidden_dim; dropout=0.0; readout='mean'

    if MODEL_NAME == 'GAT':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; n_heads=8; hidden_dim=19; out_dim=n_heads*hidden_dim; dropout=0.0; readout='mean'
        print('True hidden dim:',out_dim)

    if MODEL_NAME == 'GraphSage':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=108; out_dim=hidden_dim; dropout=0.0; readout='mean'

    if MODEL_NAME == 'MLP':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        gated=False; # MEAN
        L=4; hidden_dim=150; out_dim=hidden_dim; dropout=0.0; readout='mean'
        gated=True; # GATED
        L=4; hidden_dim=135; out_dim=hidden_dim; dropout=0.0; readout='mean'
        
    if MODEL_NAME == 'DiffPool':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=32; out_dim=hidden_dim; dropout=0.0; readout='mean'
        n_heads=8; gnn_per_block=3; embedding_dim=32; batch_size=128; pool_ratio=0.15

    if MODEL_NAME == 'GIN':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=110; out_dim=hidden_dim; dropout=0.0; readout='mean'
        n_mlp_GIN = 2; learn_eps_GIN=True; neighbor_aggr_GIN='sum'

    if MODEL_NAME == 'MoNet':
        seed=41; epochs=1000; batch_size=50; init_lr=5e-4; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=4; hidden_dim=90; out_dim=hidden_dim; dropout=0.0; readout='mean'
        pseudo_dim_MoNet=2; kernel=3;
        
    if MODEL_NAME == 'RingGNN':
        seed=41; epochs=1000; batch_size=1; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        #L=4; hidden_dim=145; out_dim=hidden_dim; dropout=0.0; readout='mean'
        L=4; hidden_dim=25; out_dim=hidden_dim; dropout=0.0
        
    if MODEL_NAME == '3WLGNN':
        seed=41; epochs=1000; batch_size=1; init_lr=5e-5; lr_reduce_factor=0.5; lr_schedule_patience=25; min_lr = 1e-6; weight_decay=0
        L=3; hidden_dim=82; out_dim=hidden_dim; dropout=0.0
      

        
    # generic new_params
    net_params = {}
    net_params['device'] = device
    net_params['in_dim'] = torch.unique(trainset[0][0].ndata['feat'],dim=0).size(0) # node_dim (feat is an integer)
    net_params['hidden_dim'] = hidden_dim
    net_params['out_dim'] = out_dim
    num_classes = torch.unique(trainset[0][1],dim=0).size(0)
    net_params['n_classes'] = num_classes
    net_params['L'] = L  # min L should be 2
    net_params['readout'] = "mean"
    net_params['layer_norm'] = True
    net_params['batch_norm'] = True
    net_params['in_feat_dropout'] = 0.0
    net_params['dropout'] = 0.0
    net_params['residual'] = True
    net_params['edge_feat'] = edge_feat
    net_params['self_loop'] = self_loop
    
    # for MLPNet 
    net_params['gated'] = gated  
    
    # for GAT
    net_params['n_heads'] = n_heads 
    
    # for graphsage
    net_params['sage_aggregator'] = 'meanpool' 

    # specific for GIN
    net_params['n_mlp_GIN'] = n_mlp_GIN
    net_params['learn_eps_GIN'] = True
    net_params['neighbor_aggr_GIN'] = 'sum'
    
    # specific for MoNet
    net_params['pseudo_dim_MoNet'] = pseudo_dim_MoNet
    net_params['kernel'] = kernel
    
    # specific for RingGNN
    net_params['radius'] = 2
    num_nodes = [dataset.train[i][0].number_of_nodes() for i in range(len(dataset.train))]
    net_params['avg_node_num'] = int(np.ceil(np.mean(num_nodes)))
    
    # specific for 3WLGNN
    net_params['depth_of_mlp'] = 2
    
    # specific for pos_enc_dim
    net_params['pos_enc'] = pos_enc
    net_params['pos_enc_dim'] = pos_enc_dim
    

In [19]:
"""
    VIEWING MODEL CONFIG AND PARAMS
"""
def view_model_param(MODEL_NAME, net_params):
    model = gnn_model(MODEL_NAME, net_params)
    total_param = 0
    print("MODEL DETAILS:\n")
    print(model)
    for param in model.parameters():
        # print(param.data.size())
        total_param += np.prod(list(param.data.size()))
    print('MODEL/Total parameters:', MODEL_NAME, total_param)
    return total_param


if notebook_mode == True:
    view_model_param(MODEL_NAME, net_params)

MODEL DETAILS:

GatedGCNNet(
  (embedding_pos_enc): Linear(in_features=10, out_features=70, bias=True)
  (embedding_h): Embedding(7, 70)
  (embedding_e): Linear(in_features=1, out_features=70, bias=True)
  (layers): ModuleList(
    (0): GatedGCNLayer(in_channels=70, out_channels=70)
    (1): GatedGCNLayer(in_channels=70, out_channels=70)
    (2): GatedGCNLayer(in_channels=70, out_channels=70)
    (3): GatedGCNLayer(in_channels=70, out_channels=70)
    (4): GatedGCNLayer(in_channels=70, out_channels=70)
    (5): GatedGCNLayer(in_channels=70, out_channels=70)
    (6): GatedGCNLayer(in_channels=70, out_channels=70)
    (7): GatedGCNLayer(in_channels=70, out_channels=70)
    (8): GatedGCNLayer(in_channels=70, out_channels=70)
    (9): GatedGCNLayer(in_channels=70, out_channels=70)
    (10): GatedGCNLayer(in_channels=70, out_channels=70)
    (11): GatedGCNLayer(in_channels=70, out_channels=70)
    (12): GatedGCNLayer(in_channels=70, out_channels=70)
    (13): GatedGCNLayer(in_channels=70, o

In [20]:
"""
    TRAINING CODE
"""

def train_val_pipeline(MODEL_NAME, dataset, params, net_params, dirs):
    
    start0 = time.time()
    per_epoch_time = []
    
    DATASET_NAME = dataset.name
    
    if MODEL_NAME in ['GCN', 'GAT']:
        if net_params['self_loop']:
            print("[!] Adding graph self-loops for GCN/GAT models (central node trick).")
            dataset._add_self_loops()
    
    if MODEL_NAME in ['GatedGCN']:
        if net_params['pos_enc']:
            print("[!] Adding graph positional encoding.")
            dataset._add_positional_encodings(net_params['pos_enc_dim'])
            print('Time PE:',time.time()-start0)
        
    trainset, valset, testset = dataset.train, dataset.val, dataset.test
        
    root_log_dir, root_ckpt_dir, write_file_name, write_config_file = dirs
    device = net_params['device']
    
    # Write network and optimization hyper-parameters in folder config/
    with open(write_config_file + '.txt', 'w') as f:
        f.write("""Dataset: {},\nModel: {}\n\nparams={}\n\nnet_params={}\n\n\nTotal Parameters: {}\n\n"""\
                .format(DATASET_NAME, MODEL_NAME, params, net_params, net_params['total_param']))
        
    log_dir = os.path.join(root_log_dir, "RUN_" + str(0))
    writer = SummaryWriter(log_dir=log_dir)

    # setting seeds
    random.seed(params['seed'])
    np.random.seed(params['seed'])
    torch.manual_seed(params['seed'])
    if device.type == 'cuda':
        torch.cuda.manual_seed(params['seed'])
    
    print("Training Graphs: ", len(trainset))
    print("Validation Graphs: ", len(valset))
    print("Test Graphs: ", len(testset))
    print("Number of Classes: ", net_params['n_classes'])

    model = gnn_model(MODEL_NAME, net_params)
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=params['init_lr'], weight_decay=params['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=params['lr_reduce_factor'],
                                                     patience=params['lr_schedule_patience'],
                                                     verbose=True)
    
    epoch_train_losses, epoch_val_losses = [], []
    epoch_train_accs, epoch_val_accs = [], [] 
    
    if MODEL_NAME in ['RingGNN', '3WLGNN']:
        # import train functions specific for WL-GNNs
        from train.train_SBMs_node_classification import train_epoch_dense as train_epoch, evaluate_network_dense as evaluate_network
        
        train_loader = DataLoader(trainset, shuffle=True, collate_fn=dataset.collate_dense_gnn)
        val_loader = DataLoader(valset, shuffle=False, collate_fn=dataset.collate_dense_gnn)
        test_loader = DataLoader(testset, shuffle=False, collate_fn=dataset.collate_dense_gnn)
        
    else:
        # import train functions for all other GCNs
        from train.train_SBMs_node_classification import train_epoch_sparse as train_epoch, evaluate_network_sparse as evaluate_network # import train functions
        
        train_loader = DataLoader(trainset, batch_size=params['batch_size'], shuffle=True, collate_fn=dataset.collate)
        val_loader = DataLoader(valset, batch_size=params['batch_size'], shuffle=False, collate_fn=dataset.collate)
        test_loader = DataLoader(testset, batch_size=params['batch_size'], shuffle=False, collate_fn=dataset.collate)
        
    # At any point you can hit Ctrl + C to break out of training early.
    try:
        with tqdm(range(params['epochs'])) as t:
            for epoch in t:

                t.set_description('Epoch %d' % epoch)

                start = time.time()

                if MODEL_NAME in ['RingGNN', '3WLGNN']: # since different batch training function for dense GNNs
                    epoch_train_loss, epoch_train_acc, optimizer = train_epoch(model, optimizer, device, train_loader, epoch, params['batch_size'])
                else:   # for all other models common train function
                    epoch_train_loss, epoch_train_acc, optimizer = train_epoch(model, optimizer, device, train_loader, epoch)
                    
                epoch_val_loss, epoch_val_acc = evaluate_network(model, device, val_loader, epoch)
                _, epoch_test_acc = evaluate_network(model, device, test_loader, epoch)        
                
                epoch_train_losses.append(epoch_train_loss)
                epoch_val_losses.append(epoch_val_loss)
                epoch_train_accs.append(epoch_train_acc)
                epoch_val_accs.append(epoch_val_acc)

                writer.add_scalar('train/_loss', epoch_train_loss, epoch)
                writer.add_scalar('val/_loss', epoch_val_loss, epoch)
                writer.add_scalar('train/_acc', epoch_train_acc, epoch)
                writer.add_scalar('val/_acc', epoch_val_acc, epoch)
                writer.add_scalar('test/_acc', epoch_test_acc, epoch)
                writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], epoch)

                t.set_postfix(time=time.time()-start, lr=optimizer.param_groups[0]['lr'],
                              train_loss=epoch_train_loss, val_loss=epoch_val_loss,
                              train_acc=epoch_train_acc, val_acc=epoch_val_acc,
                              test_acc=epoch_test_acc)

                per_epoch_time.append(time.time()-start)

                # Saving checkpoint
                ckpt_dir = os.path.join(root_ckpt_dir, "RUN_")
                if not os.path.exists(ckpt_dir):
                    os.makedirs(ckpt_dir)
                torch.save(model.state_dict(), '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch)))

                files = glob.glob(ckpt_dir + '/*.pkl')
                for file in files:
                    epoch_nb = file.split('_')[-1]
                    epoch_nb = int(epoch_nb.split('.')[0])
                    if epoch_nb < epoch-1:
                        os.remove(file)

                scheduler.step(epoch_val_loss)

                if optimizer.param_groups[0]['lr'] < params['min_lr']:
                    print("\n!! LR SMALLER OR EQUAL TO MIN LR THRESHOLD.")
                    break
                    
                # Stop training after params['max_time'] hours
                if time.time()-start0 > params['max_time']*3600:
                    print('-' * 89)
                    print("Max_time for training elapsed {:.2f} hours, so stopping".format(params['max_time']))
                    break
    
    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early because of KeyboardInterrupt')
    
    
    _, test_acc = evaluate_network(model, device, test_loader, epoch)
    _, train_acc = evaluate_network(model, device, train_loader, epoch)
    print("Test Accuracy: {:.4f}".format(test_acc))
    print("Train Accuracy: {:.4f}".format(train_acc))
    print("Convergence Time (Epochs): {:.4f}".format(epoch))
    print("TOTAL TIME TAKEN: {:.4f}s".format(time.time()-start0))
    print("AVG TIME PER EPOCH: {:.4f}s".format(np.mean(per_epoch_time)))

    writer.close()

    """
        Write the results in out_dir/results folder
    """
    with open(write_file_name + '.txt', 'w') as f:
        f.write("""Dataset: {},\nModel: {}\n\nparams={}\n\nnet_params={}\n\n{}\n\nTotal Parameters: {}\n\n
    FINAL RESULTS\nTEST ACCURACY: {:.4f}\nTRAIN ACCURACY: {:.4f}\n\n
    Convergence Time (Epochs): {:.4f}\nTotal Time Taken: {:.4f} hrs\nAverage Time Per Epoch: {:.4f} s\n\n\n"""\
          .format(DATASET_NAME, MODEL_NAME, params, net_params, model, net_params['total_param'],
                  test_acc, train_acc, epoch, (time.time()-start0)/3600, np.mean(per_epoch_time)))

        

In [21]:
!pip install nbconvert
import nbconvert

In [ ]:
def main(notebook_mode=False,config=None):
    
    """
        USER CONTROLS
    """
    
    # terminal mode
    if notebook_mode==False:
        
        parser = argparse.ArgumentParser()
        parser.add_argument('--config', help="Please give a config.json file with training/model/data/param details")
        parser.add_argument('--gpu_id', help="Please give a value for gpu id")
        parser.add_argument('--model', help="Please give a value for model name")
        parser.add_argument('--dataset', help="Please give a value for dataset name")
        parser.add_argument('--out_dir', help="Please give a value for out_dir")
        parser.add_argument('--seed', help="Please give a value for seed")
        parser.add_argument('--epochs', help="Please give a value for epochs")
        parser.add_argument('--batch_size', help="Please give a value for batch_size")
        parser.add_argument('--init_lr', help="Please give a value for init_lr")
        parser.add_argument('--lr_reduce_factor', help="Please give a value for lr_reduce_factor")
        parser.add_argument('--lr_schedule_patience', help="Please give a value for lr_schedule_patience")
        parser.add_argument('--min_lr', help="Please give a value for min_lr")
        parser.add_argument('--weight_decay', help="Please give a value for weight_decay")
        parser.add_argument('--print_epoch_interval', help="Please give a value for print_epoch_interval")    
        parser.add_argument('--L', help="Please give a value for L")
        parser.add_argument('--hidden_dim', help="Please give a value for hidden_dim")
        parser.add_argument('--out_dim', help="Please give a value for out_dim")
        parser.add_argument('--residual', help="Please give a value for residual")
        parser.add_argument('--edge_feat', help="Please give a value for edge_feat")
        parser.add_argument('--readout', help="Please give a value for readout")
        parser.add_argument('--kernel', help="Please give a value for kernel")
        parser.add_argument('--n_heads', help="Please give a value for n_heads")
        parser.add_argument('--gated', help="Please give a value for gated")
        parser.add_argument('--in_feat_dropout', help="Please give a value for in_feat_dropout")
        parser.add_argument('--dropout', help="Please give a value for dropout")
        parser.add_argument('--layer_norm', help="Please give a value for layer_norm")
        parser.add_argument('--batch_norm', help="Please give a value for batch_norm")
        parser.add_argument('--sage_aggregator', help="Please give a value for sage_aggregator")
        parser.add_argument('--data_mode', help="Please give a value for data_mode")
        parser.add_argument('--num_pool', help="Please give a value for num_pool")
        parser.add_argument('--gnn_per_block', help="Please give a value for gnn_per_block")
        parser.add_argument('--embedding_dim', help="Please give a value for embedding_dim")
        parser.add_argument('--pool_ratio', help="Please give a value for pool_ratio")
        parser.add_argument('--linkpred', help="Please give a value for linkpred")
        parser.add_argument('--cat', help="Please give a value for cat")
        parser.add_argument('--self_loop', help="Please give a value for self_loop")
        parser.add_argument('--max_time', help="Please give a value for max_time")
        parser.add_argument('--pos_enc_dim', help="Please give a value for pos_enc_dim")
        parser.add_argument('--pos_enc', help="Please give a value for pos_enc")
        args = parser.parse_args()
        with open(args.config) as f:
            config = json.load(f)
            

        # device
        if args.gpu_id is not None:
            config['gpu']['id'] = int(args.gpu_id)
            config['gpu']['use'] = True
        device = gpu_setup(config['gpu']['use'], config['gpu']['id'])

        # model, dataset, out_dir
        if args.model is not None:
            MODEL_NAME = args.model
        else:
            MODEL_NAME = config['model']
        if args.dataset is not None:
            DATASET_NAME = args.dataset
        else:
            DATASET_NAME = config['dataset']
        dataset = LoadData(DATASET_NAME)
        if args.out_dir is not None:
            out_dir = args.out_dir
        else:
            out_dir = config['out_dir']

        # parameters
        params = config['params']
        if args.seed is not None:
            params['seed'] = int(args.seed)
        if args.epochs is not None:
            params['epochs'] = int(args.epochs)
        if args.batch_size is not None:
            params['batch_size'] = int(args.batch_size)
        if args.init_lr is not None:
            params['init_lr'] = float(args.init_lr)
        if args.lr_reduce_factor is not None:
            params['lr_reduce_factor'] = float(args.lr_reduce_factor)
        if args.lr_schedule_patience is not None:
            params['lr_schedule_patience'] = int(args.lr_schedule_patience)
        if args.min_lr is not None:
            params['min_lr'] = float(args.min_lr)
        if args.weight_decay is not None:
            params['weight_decay'] = float(args.weight_decay)
        if args.print_epoch_interval is not None:
            params['print_epoch_interval'] = int(args.print_epoch_interval)
        if args.max_time is not None:
            params['max_time'] = float(args.max_time)

        # network parameters
        net_params = config['net_params']
        net_params['device'] = device
        net_params['gpu_id'] = config['gpu']['id']
        net_params['batch_size'] = params['batch_size']
        if args.L is not None:
            net_params['L'] = int(args.L)
        if args.hidden_dim is not None:
            net_params['hidden_dim'] = int(args.hidden_dim)
        if args.out_dim is not None:
            net_params['out_dim'] = int(args.out_dim)   
        if args.residual is not None:
            net_params['residual'] = True if args.residual=='True' else False
        if args.edge_feat is not None:
            net_params['edge_feat'] = True if args.edge_feat=='True' else False
        if args.readout is not None:
            net_params['readout'] = args.readout
        if args.kernel is not None:
            net_params['kernel'] = int(args.kernel)
        if args.n_heads is not None:
            net_params['n_heads'] = int(args.n_heads)
        if args.gated is not None:
            net_params['gated'] = True if args.gated=='True' else False
        if args.in_feat_dropout is not None:
            net_params['in_feat_dropout'] = float(args.in_feat_dropout)
        if args.dropout is not None:
            net_params['dropout'] = float(args.dropout)
        if args.layer_norm is not None:
            net_params['layer_norm'] = True if args.layer_norm=='True' else False
        if args.batch_norm is not None:
            net_params['batch_norm'] = True if args.batch_norm=='True' else False
        if args.sage_aggregator is not None:
            net_params['sage_aggregator'] = args.sage_aggregator
        if args.data_mode is not None:
            net_params['data_mode'] = args.data_mode
        if args.num_pool is not None:
            net_params['num_pool'] = int(args.num_pool)
        if args.gnn_per_block is not None:
            net_params['gnn_per_block'] = int(args.gnn_per_block)
        if args.embedding_dim is not None:
            net_params['embedding_dim'] = int(args.embedding_dim)
        if args.pool_ratio is not None:
            net_params['pool_ratio'] = float(args.pool_ratio)
        if args.linkpred is not None:
            net_params['linkpred'] = True if args.linkpred=='True' else False
        if args.cat is not None:
            net_params['cat'] = True if args.cat=='True' else False
        if args.self_loop is not None:
            net_params['self_loop'] = True if args.self_loop=='True' else False
        if args.pos_enc is not None:
            net_params['pos_enc'] = True if args.pos_enc=='True' else False
        if args.pos_enc_dim is not None:
            net_params['pos_enc_dim'] = int(args.pos_enc_dim)

            
    # notebook mode
    if notebook_mode:
        
        # parameters
        params = config['params']
        
        # dataset
        DATASET_NAME = config['dataset']
        dataset = LoadData(DATASET_NAME)
        
        # device
        device = gpu_setup(config['gpu']['use'], config['gpu']['id'])
        out_dir = config['out_dir']
        
        # GNN model
        MODEL_NAME = config['model']
        
        # network parameters
        net_params = config['net_params']
        net_params['device'] = device
        net_params['gpu_id'] = config['gpu']['id']
        net_params['batch_size'] = params['batch_size']
        
            
    # SBM
    net_params['in_dim'] = torch.unique(dataset.train[0][0].ndata['feat'],dim=0).size(0) # node_dim (feat is an integer)
    net_params['n_classes'] = torch.unique(dataset.train[0][1],dim=0).size(0)
    
    if MODEL_NAME == 'RingGNN':
        num_nodes = [dataset.train[i][0].number_of_nodes() for i in range(len(dataset.train))]
        net_params['avg_node_num'] = int(np.ceil(np.mean(num_nodes)))

    root_log_dir = out_dir + 'logs/' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" + str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    root_ckpt_dir = out_dir + 'checkpoints/' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" + str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    write_file_name = out_dir + 'results/result_' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" + str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    write_config_file = out_dir + 'configs/config_' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" + str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    dirs = root_log_dir, root_ckpt_dir, write_file_name, write_config_file

    if not os.path.exists(out_dir + 'results'):
        os.makedirs(out_dir + 'results')
        
    if not os.path.exists(out_dir + 'configs'):
        os.makedirs(out_dir + 'configs')

    net_params['total_param'] = view_model_param(MODEL_NAME, net_params)
    train_val_pipeline(MODEL_NAME, dataset, params, net_params, dirs)

    
    
    
    
    
if notebook_mode==True:
    
    config = {}
    # gpu config
    gpu = {}
    gpu['use'] = use_gpu
    gpu['id'] = gpu_id
    config['gpu'] = gpu
    # GNN model, dataset, out_dir
    config['model'] = MODEL_NAME
    config['dataset'] = DATASET_NAME
    config['out_dir'] = out_dir
    # parameters
    params = {}
    params['seed'] = seed
    params['epochs'] = epochs
    params['batch_size'] = batch_size
    params['init_lr'] = init_lr
    params['lr_reduce_factor'] = lr_reduce_factor 
    params['lr_schedule_patience'] = lr_schedule_patience
    params['min_lr'] = min_lr
    params['weight_decay'] = weight_decay
    params['print_epoch_interval'] = 5
    params['max_time'] = max_time
    config['params'] = params
    # network parameters
    config['net_params'] = net_params
    
    # convert to .py format
    from utils.cleaner_main import *
    cleaner_main('main_SBMs_node_classification')
    
    main(True,config)
    
else:
    
    main()
    

Convert main_SBMs_node_classification.ipynb to main_SBMs_node_classification.py
Clean main_SBMs_node_classification.py
Done. 
[I] Loading dataset SBM_CLUSTER...
train, test, val sizes : 10000 1000 1000
[I] Finished loading.
[I] Data load time: 12.8254s
cuda available with GPU: Tesla P100-PCIE-16GB
MODEL DETAILS:

GatedGCNNet(
  (embedding_pos_enc): Linear(in_features=10, out_features=70, bias=True)
  (embedding_h): Embedding(7, 70)
  (embedding_e): Linear(in_features=1, out_features=70, bias=True)
  (layers): ModuleList(
    (0): GatedGCNLayer(in_channels=70, out_channels=70)
    (1): GatedGCNLayer(in_channels=70, out_channels=70)
    (2): GatedGCNLayer(in_channels=70, out_channels=70)
    (3): GatedGCNLayer(in_channels=70, out_channels=70)
    (4): GatedGCNLayer(in_channels=70, out_channels=70)
    (5): GatedGCNLayer(in_channels=70, out_channels=70)
    (6): GatedGCNLayer(in_channels=70, out_channels=70)
    (7): GatedGCNLayer(in_channels=70, out_channels=70)
    (8): GatedGCNLayer(in

/usr/local/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.adjacency_matrix_scipy is deprecated. Please replace it with:

	DGLGraph.adjacency_matrix(transpose, scipy_fmt="csr").

  return warnings.warn(message, category=category, stacklevel=1)
/content/benchmarking-gnns/data/SBMs.py:143: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:240.)
  g.ndata['pos_enc'] = torch.from_numpy(EigVec[:,1:pos_enc_dim+1]).float()


Time PE: 82.18104219436646
Training Graphs:  10000
Validation Graphs:  1000
Test Graphs:  1000
Number of Classes:  6


Epoch 84:   8%|▊         | 84/1000 [4:55:57<53:49:43, 211.55s/it, lr=5e-5, test_acc=74.1, time=212, train_acc=79.4, train_loss=0.547, val_acc=74.2, val_loss=0.705]

Epoch    84: reducing learning rate of group 0 to 2.5000e-05.


...

Epoch 136:  14%|█▎        | 136/1000 [7:57:37<50:10:16, 209.05s/it, lr=1.25e-5, test_acc=75.2, time=211, train_acc=81.6, train_loss=0.49, val_acc=75.4, val_loss=0.688]

Epoch   136: reducing learning rate of group 0 to 6.2500e-06.


Epoch 162:  16%|█▌        | 162/1000 [9:26:15<47:07:39, 202.46s/it, lr=6.25e-6, test_acc=75.3, time=203, train_acc=82, train_loss=0.481, val_acc=75.6, val_loss=0.691]

Epoch   162: reducing learning rate of group 0 to 3.1250e-06.


Epoch 170:  17%|█▋        | 170/1000 [9:52:59<46:03:32, 199.77s/it, lr=3.13e-6, test_acc=75.3, time=199, train_acc=82.1, train_loss=0.479, val_acc=75.5, val_loss=0.694]